In [ ]:
# plt.show()で可視化されない人はこのセルを実行してください。
%matplotlib inline

In [ ]:
# 初めの一回だけこのセルを実行してください、データセットをダウンロードして展開します
# 一回実行すれば、データセットはダウンロードされたままなので、再起動後等再び実行する必要はありません
import urllib.request
import zipfile

# URLを指定
url = "https://storage.googleapis.com/tutor-contents-dataset/5030_unsupervised_learning_data.zip"
save_name = url.split('/')[-1]

# ダウンロードする
mem = urllib.request.urlopen(url).read()

# ファイルへ保存
with open(save_name, mode='wb') as f:
    f.write(mem)

# zipファイルをカレントディレクトリに展開する
zfile = zipfile.ZipFile(save_name)
zfile.extractall('.')

# 総合添削問題

画像の色を減らし、N色で構成された画像に変換することを考えます。このような場合、クラスタリングによりこの問題を解決することが可能です。
<br>
<br>
まずは単純な問題として、青空と海が写った青の多い画像を2色に減色することを考えましょう。
<br>
画像の画素はR（赤）, G（緑）, B（青）の3色で表されます。
<br>
R, G, Bを横（x軸）、縦（y軸）、高さ（z軸）として画素を3次元空間上にマッピングすると、この画像は青側に偏った分布になります。さらに、青空と海では青の種類が違いますから、3次元空間上では画素の点は「青空の青」と「海の青」の2つの塊になっていることが予想されます。
<br>
この画像を2色に減色するには、3次元空間に分布している画素を2つにクラスタリングし、「青空の青」「海の青」それぞれの塊の中心を代表色として選べば良いことになります。<br>

<img src="https://aidemyexstorage.blob.core.windows.net/aidemycontents/1555132640882224.png" width="40%">

<br>
<br>
次に、以下の画像を減色することを考えます。
<img src="./img_cluster/img_1_0.png">

　上記の画像を32色にまで減色すると以下のようになります。

<img src="./img_cluster/output32.bmp">

<br>
具体的な流れは、以下のようになります。<br>
1. 画像の中で代表色をk-means法を用いてN種類選ぶ。<br>
2. 画像中の全ピクセルを一番色が近い代表色に置き換える。 (ex. 藍色: RGB値(19, 74, 99)→青色: RGB値(0, 0, 255))<br>
(2.における色の近さの定義は、画像中の色は(R, G, B)=(それぞれ0~255)で表されるため、RGBの値の大きさの3次元のユークリッド距離で算出されます。)<br>

　　　　 
       $(R1,G1,B1)と(R2,G2,B2)の距離= \sqrt{(R1−R2)^2+(G1−G2)^2+(B1−B2)^2}$
 
はじめに、ランダムにN個の代表色を選択します。<br>
画像上の全ピクセルにおいて、一番近い代表色を選び、全ピクセルをN個のクラスターに割り当てます。<br>
それぞれのクラスターにおける平均色(R,G,Bのそれぞれの平均値をとった値)を算出します。<br>
新しい代表色を用いて、再度2.を繰り返します。<br>
これを繰り返すと、各クラスターの平均値が代表色の値と一致し、代表色の変化が止まります。これを最終的な代表色として割り当てます。<br>

#### 問題

- 上記のアルゴリズムを理解し、画像とNを入力されると、画像をN色で構成される画像に変換するプログラムを構築してください。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import randint
from PIL import Image
import cv2

# ------------#
# パラメータ #
# ------------#
# 減色後の色数（任意の個数の色数を指定できます）
Colors = [32]


# k平均法による減色処理
def run_kmeans(pixels, k):
    # 代表色の番号を定義。(pixelsの要素数の配列を0で初期化)
    cls = [0] * len(pixels)

    # 代表色の初期値をランダムに設定
    center = [randint(256), randint(256), randint(256)]
    # randintにより[R, G, B]をそれぞれ0~255の範囲でk個生成
    for i in range(k):
        center.append(np.array([randint(256), randint(256), randint(256)]))

    # ねじれの回数を定義
    distortion = 0

    # 最大50回のIterationを実施
    for iter_num in range(50):
        # 各代表色のRBG値を保存
        center_new = []
        for i in range(k):
            center_new.append(np.array([0, 0, 0]))

        num_points = [0] * k
        distortion_new = 0

        # 各データが属するグループ（代表色）を計算
        for pix, point in enumerate(pixels):
            # 最小ユークリッド距離を定義してください
            min_dist = 3*(255**2)
            point = np.array(point)
            for i in range(k):
                # ピクセルの座標におけるRGB値と代表色のユークリッド距離の2乗を算出
                d = sum([x * x for x in point - center[i]])
                # dの値が設定した最小値より小さければ最小値にdを代入してください
                # 各ピクセルの代表色の番号をリストcls[pix]に保存してください
                if d < min_dist:
                    min_dist = d
                    cls[pix] = i
                else:
                    continue;
            # 代表色のRGB値を更新してください
            center_new[cls[pix]] += [point[0], point[1], point[2]]
            num_points[cls[pix]] += 1 
            # 最小値の更新を記録してください
            distortion_new = min_dist

        # 新しい代表色を計算
        for i in range(k):
            center_new[i] = center_new[i] / num_points[i]
        center = center_new
        print("Distortion = %d" % distortion_new)

        # Distortion(J)の変化が0.5%未満になったら終了
        if iter_num > 0 and distortion - distortion_new < distortion * 0.005:
            break
        distortion = distortion_new

    # 画像データの各ピクセルを代表色のRGB値で置き換え
    for pix, point in enumerate(pixels):
        pixels[pix] = tuple(map(lambda x: int(x), center[cls[pix]]))

    return pixels


for k in Colors:
    print("k=%d" % k)
    # 画像ファイルの読み込み
    im = Image.open("./img_cluster/img_1_0.png")
    pixels = list(im.convert('RGB').getdata())
    # k平均法による減色処理
    result = run_kmeans(pixels, k)
    # 画像データの更新とファイル出力
    im.putdata(result)  # Update image
    im.save("output%02d.bmp" % k, "BMP")

#### ヒント

- `(取りうる最大のユークリッド距離の2乗)`は `(RGBの3種類) × (値の最大値の2乗)` で計算できます
- `pixels`は`[(R, G, B), ... , (R, G, B)]`という風にtuple型の各座標のRGB値を内包した配列です
- `enumerate()`をfor文で用いることでインデックスと要素を同時に扱うことができます

##  解答例

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import randint
from PIL import Image
import cv2

#------------#
# パラメータ #
#------------#
# 減色後の色数（任意の個数の色数を指定できます）
Colors = [32]


# k平均法による減色処理
def run_kmeans(pixels, k):
    cls = [0] * len(pixels)    

    # 代表色の初期値をランダムに設定
    center = []
    # randintにより[R, G, B]をそれぞれ0~255の範囲でk個生成
    for i in range(k):
        center.append(np.array([randint(256), randint(256), randint(256)]))
    # numpy.arrayをlist型に変換
    print (map(lambda x: x.tolist(), center))
    # ねじれの回数を定義
    distortion = 0

    # 最大50回のIterationを実施
    for iter_num in range(50):
        # 各代表色のRBG値を保存
        center_new = []
        for i in range(k):
            center_new.append(np.array([0,0,0]))

        num_points = [0] * k
        distortion_new = 0

        # E Phase: 各データが属するグループ（代表色）を計算
        for pix, point in enumerate(pixels):
            min_dist = 256*256*3
            point = np.array(point)
            for i in range(k):
                # ピクセルの座標におけるRGB値と代表色のユークリッド距離を算出
                d = sum([x*x for x in point-center[i]])
                # dの値が設定した最小値より小さければ最小値にdを代入し、
                # 各ピクセルの代表色の番号をリストclsに保存
                if d < min_dist:
                    min_dist = d
                    cls[pix] = i
            # 代表色のRGB値を更新
            center_new[cls[pix]] += point
            #
            num_points[cls[pix]] += 1    # 代表色の更新回数を更新
            # 最小値の更新を記録
            distortion_new += min_dist

        # M Phase: 新しい代表色を計算
        for i in range(k):
            center_new[i] = center_new[i] / num_points[i]
        center = center_new
        print (map(lambda x: x.tolist(), center))
        print ("Distortion = %d" % distortion_new)

        # Distortion(J)の変化が0.5%未満になったら終了
        if iter_num > 0 and distortion - distortion_new < distortion * 0.005:
            break
        distortion = distortion_new

    # 画像データの各ピクセルを代表色で置き換え
    for pix, point in enumerate(pixels):
        pixels[pix] = tuple(map(lambda x: int(x), center[cls[pix]]))

    return pixels


for k in Colors:
    print ("k=%d" % k)
    # 画像ファイルの読み込み
    im = Image.open("./img_cluster/img_1_0.png")
    pixels = list(im.convert('RGB').getdata())
    # k平均法による減色処理
    result = run_kmeans(pixels, k)
    # 画像データの更新とファイル出力
    im.putdata(result) # Update image
    im.save("output%02d.bmp" % k, "BMP")

ModuleNotFoundError: No module named 'cv2'

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import randint
from PIL import Image
import cv2

#---------#
# pixels:各点のRGB値をタプルで並べたもの
# cls:各点がどの代表色に属するのかを記録
# center:代表色のRGB値をタプルで並べたもの
# distortion:現時点での点と代表点の最小距離（最小距離の更新率の計算に使用）
# center_new:代表色の更新に用いる
# num_points:各代表色に何点属するのかを数えたもの
# min_dist:ある点pと各代表点の距離の一番小さいやつ（つまりその最小距離の代表点に点pは属する）
# min_distの初期値は最大距離を設定します256の2乗*3
# distortion_new:現時点での点と代表点の最小距離の更新に用いる
# cls[pix]:その点がどの代表点に属するか
#---------#

# k平均法による減色処理
def run_kmeans(pixels, k):
    # 各ピクセルの代表色番号を格納するためのリストcls(長さ:ピクセル数、0で初期化)
    cls = [0] * len(pixels)

    # 代表色の初期値をランダムに設定
    center = []
    # randintにより[R, G, B]をそれぞれ0~255の範囲でk個生成
    for i in range(k):
        center.append(np.array([randint(256), randint(256), randint(256)]))
        
    # numpy.arrayをlist型に変換
    center = list(center)
    print (map(lambda x: x.tolist(), center))
    # ねじれの回数を定義
    distortion = 0
    
    # 最大50回のIterationを実施
    for iter_num in range(50):
        # 各代表色のRBG値を保存
        
        # 更新後の代表値を格納するためのリストcenter_newを宣言し、[0 0 0]で初期化
        center_new = []    
        for i in range(k):
            center_new.append(np.array([0,0,0]))

        num_points = [0] * k
        distortion_new = 0
        
        # 各データが属するグループ（代表色）を計算
        for pix, point in enumerate(pixels):
            # 最小ユークリッド距離を定義してください
            min_dist = 3*(256**2)
            point = np.array(point)   # 対象ピクセルの座標をpointに再代入
            # 対象ピクセル(point)と各代表色間のユークリッド距離を計算し、min_distを決定
            for i in range(k):
                d = sum([x**x for x in point-center[i]])
                # dが最小値の場合、min_distに代入
                if d < min_dist:
                    min_dist = d
                    cls[pix] = i    # min_distが更新される度に、iの値は変化
            # 代表色のRGB値を更新
            center_new[cls[pix]] += point   # 特定の代表色にRGBの値を足していく
            num_points[cls[pix]] += 1      # 特定の代表色に割り当てた回数をそれぞれカウント
            # 最小値の更新を記録
            distortion_new += min_dist    # 代表値との距離の格納
            
            # M Phase: 新しい代表色を計算
            for i in range(k):
                # 新しい代表色＝クラスタ毎に割り当てたピクセルの平均値を算出
                center_new[i] = center_new[i] / num_points[i]
            center = center_new
            print (map(lambda x: x.tolist(), center))
            print ("Distortion = %d" % distortion_new)
        
            # Distortion(J)の変化が0.5%未満になったら終了
            dist_delt = distortion - distortion_new
            if iter_num > 0 and dist_delt < distortion * 0.005:
                break
            distortion = distortion_new
        
for k in Colors:
    print ("k=%d" % k)
    # 画像ファイルの読み込み
    im = Image.open("./img_cluster/img_1_0.png")
    pixels = list(im.convert('RGB').getdata())
    # k平均法による減色処理
    result = run_kmeans(pixels, k)
    # 画像データの更新とファイル出力
    im.putdata(result) # Update image
    im.save("output%02d.bmp" % k, "BMP")

ModuleNotFoundError: No module named 'cv2'

添削課題の提出は以下のアドレスから提出いただきますようお願いします。<br>

https://goo.gl/forms/fW7CAspZMwHuWuqk2<br><br>
以下のアドレスからアンケートにご協力頂きたく存じます。<br>
ご回答のほど、よろしくお願いいたします。

https://goo.gl/forms/WHjJQYeodIndRvyz2